<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2007%20-%20Chains_and_Why_They_Are_Used.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# Calling

## __ call __

In [2]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "What is a word to replace the following: {word}?"

# Set the "OPENAI_API_KEY" environment variable before running following line.
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain("artificial")

c:\Users\Christian\AppData\Local\pypoetry\Cache\virtualenvs\data-science-project-VmJyb5kE-py3.11\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (4.1.6) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
c:\Users\Christian\AppData\Local\pypoetry\Cache\virtualenvs\data-science-project-VmJyb5kE-py3.11\Lib\site-packages\langchain\llms\openai.py:179: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\Christian\AppData\Local\pypoetry\Cache\virtualenvs\data-science-project-VmJyb5kE-py3.11\Lib\site-packages\langchain\llms\openai.py:753: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


{'word': 'artificial', 'text': 'synthetic'}

## Apply

In [3]:
input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]

llm_chain.apply(input_list)

ValueError: OpenAIChat currently only supports single prompt, got ['What is a word to replace the following: artificial?', 'What is a word to replace the following: intelligence?', 'What is a word to replace the following: robot?']

## Generate

In [4]:
llm_chain.generate(input_list)

ValueError: OpenAIChat currently only supports single prompt, got ['What is a word to replace the following: artificial?', 'What is a word to replace the following: intelligence?', 'What is a word to replace the following: robot?']

## Predict

#### Multiple

In [ ]:
prompt_template = "Looking at the context of '{context}'. What is a approapriate word to replace the following: {word}?"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=prompt_template, input_variables=["word", "context"]))

llm_chain.predict(word="fan", context="object")

'\n\nVentilator'

In [ ]:
llm_chain.predict(word="fan", context="humans")

'\n\nAdmirer'

In [ ]:
# llm_chain.run(word="fan", context="object")

'\n\nVentilator'

### from string

In [ ]:
template = """Looking at the context of '{context}'. What is a approapriate word to replace the following: {word}?"""
llm_chain = LLMChain.from_string(llm=llm, template=template)

In [ ]:
llm_chain.predict(word="fan", context="object")

'\n\nVentilator'

# Parsers

In [6]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all possible words as substitute for 'artificial' as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=[], output_parser=output_parser))

llm_chain.predict()

'synthetic, man-made, fake, simulated, imitation, faux, ersatz, fabricated, counterfeit, pseudo'

In [7]:
llm_chain.predict_and_parse()

c:\Users\Christian\AppData\Local\pypoetry\Cache\virtualenvs\data-science-project-VmJyb5kE-py3.11\Lib\site-packages\langchain\chains\llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


['synthetic',
 'man-made',
 'fake',
 'simulated',
 'imitation',
 'faux',
 'counterfeit',
 'ersatz',
 'fabricated',
 'manufactured']

# Conversational Chain

In [8]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

template = """List all possible words as substitute for 'artificial' as comma separated.

Current conversation:
{history}

{input}"""

conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory())

conversation.predict_and_parse(input="Answer briefly. write the first 3 options.")

['synthetic', 'man-made', 'simulated']

In [ ]:
conversation.predict_and_parse(input="And the next 4?")

['Fabricated', 'Simulated', 'Automated', 'Constructed']

# Debug

In [ ]:
conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory(),
    verbose=True)

conversation.predict_and_parse(input="Answer briefly. write the first 3 options.")



> Entering new ConversationChain chain...
Prompt after formatting:
List all possible words as substitute for 'artificial' as comma separated.

Current conversation:


Answer briefly. write the first 3 options.

> Finished chain.


['Synthetic', 'Manufactured', 'Imitation']

# Sequential Chain

In [ ]:
# from langchain.chains import SimpleSequentialChain
# overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

# Custom Chain

In [9]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [10]:
prompt_1 = PromptTemplate(
    input_variables=["word"],
    template="What is the meaning of the following word '{word}'?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["word"],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("artificial")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:
Artificial means made or produced by human beings rather than occurring naturally. It can also refer to something that is not genuine or real.Synthetic
